In [6]:
import selenium
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [9]:
def get_jobs(keyword, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="/Users/laurentrisser/Documents/data_science/wecloudata/python_for_data_science/ds_salary/chromedriver", options=options)
    driver.set_window_size(1120, 1000)
    
    url = 'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"sc.keyword="+keyword+"&dropdown=0'
    #url = 'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType='
    #url = 'https://www.glassdoor.ca/Job/jobs.htm?sc.generalKeyword="' + keyword + '"%22&sc.locationSeoString=canada&locId=3&locT=N'
    #url = 'https://www.glassdoor.com/Job/canada-"' + keyword + '"-jobs-SRCH_IL.0,6_IN3_KO7,19.htm?countryRedirect=false'
    driver.get(url)
    jobs = []
    
    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(4)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(5)

        try:
            driver.find_element_by_id("prefix__icon-close-1").click() #clicking to the X.
            print("This is working!") 
            
        except NoSuchElementException:
            print("This is not working")
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:
            
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
                print("Salary youhou")
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
                print("No salary")
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."  
            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [10]:
data = get_jobs('data-scientist', 10, False)


This is working!
Progress: 0/10
Salary youhou
Progress: 1/10
Salary youhou
Progress: 2/10
Salary youhou
Progress: 3/10
Salary youhou
Progress: 4/10
Salary youhou
Progress: 5/10
Salary youhou
Progress: 6/10
Salary youhou
Progress: 7/10
Salary youhou
Progress: 8/10
Salary youhou
Progress: 9/10
Salary youhou
Progress: 10/10


In [11]:
data

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,General Applications,CA$63K-CA$95K (Glassdoor Est.),Work at one of the 50 best employers in Canada...,3.9,BCAA\n3.9,Canada,"Burnaby, Canada",1001 to 5000 employees,1906,Non-profit Organisation,Insurance Operators,Insurance,$100 to $500 million (CAD),-1
1,Compliance Specialist,CA$63K-CA$95K (Glassdoor Est.),Description\n\n\nThe Alberta Electric System O...,3.3,Alberta Electric System Operator\n3.3,Calgary,"Calgary, Canada",201 to 500 employees,2003,Non-profit Organisation,Utilities,"Oil, Gas, Energy & Utilities",Unknown / Non-Applicable,-1
2,Project CSO,CA$63K-CA$95K (Glassdoor Est.),Working closely with Corporate Safety and Site...,4.5,Matcon Civil\n4.5,Langley,"Langley, Canada",51 to 200 employees,1970,Company - Private,Building & Construction,"Building, Repair & Maintenance",$50 to $100 million (CAD),-1
3,Payments Analyst,CA$63K-CA$95K (Glassdoor Est.),"Friday, June 26, 2020\n\n| About the role\n\nT...",3.3,Payments Canada\n3.3,Ottawa,"Ottawa, Canada",51 to 200 employees,1980,Non-profit Organisation,Financial Transaction Processing,Finance,Unknown / Non-Applicable,-1
4,Quality Coordinator,CA$63K-CA$95K (Glassdoor Est.),"All together, more powerful.\n\nOur Cogent Tea...",3.9,Cogent Power\n3.9,Burlington,"Burlington, Canada",201 to 500 employees,-1,Subsidiary or Business Segment,Metal & Mineral Manufacturing,Manufacturing,$100 to $500 million (CAD),-1
5,Associate Producer,CA$63K-CA$95K (Glassdoor Est.),"Chez Rogers, nous relions les gens d’ici à un ...",3.8,Rogers Communications\n3.8,Montreal,"Toronto, Canada",10000+ employees,1960,Company - Public,Telecommunications Services,Telecommunications,$10+ billion (CAD),"TELUS, Bell, Shaw Communications"
6,Network Operations Centre Analyst,CA$63K-CA$95K (Glassdoor Est.),WESTMAN COMMUNICATIONS GROUP is seeking applic...,3.6,Westman Communications Group\n3.6,Brandon,"Brandon, Canada",51 to 200 employees,1977,Other Organisation,TV Broadcasting & Cable Networks,Media,$10 to $25 million (CAD),-1
7,Technical Support Specialist I,CA$63K-CA$95K (Glassdoor Est.),Together we’re building a company that will en...,4.5,PagerDuty\n4.5,Toronto,"San Francisco, CA",501 to 1000 employees,2009,Company - Public,Computer Hardware & Software,Information Technology,Unknown / Non-Applicable,-1
8,Implementation Specialist,CA$63K-CA$95K (Glassdoor Est.),What you can expect when joining the team\n\nI...,4.7,PetalMD\n4.7,Quebec,"Quebec, Canada",51 to 200 employees,2010,Company - Private,Enterprise Software & Network Solutions,Information Technology,Unknown / Non-Applicable,-1
9,Call Centre Collector,CA$63K-CA$95K (Glassdoor Est.),MetCredit is seeking full-time debt collectors...,4.3,MetCredit\n4.3,Edmonton,"Edmonton, Canada",201 to 500 employees,1973,Company - Private,Financial Transaction Processing,Finance,$10 to $25 million (CAD),-1
